# Home Depot Product Search Relevance
The challenge is to predict a relevance score for the provided combinations of search terms and products. To create the ground truth labels, Home Depot has crowdsourced the search/product pairs to multiple human raters.

## LabGraph Create
This notebook uses the LabGraph create machine learning iPython module. You need a personal licence to run this code.

In [1]:
import graphlab as gl

### Load data from CSV files

In [2]:
train = gl.SFrame.read_csv("../data/train.csv")

[INFO] This non-commercial license of GraphLab Create is assigned to thomasv1000@hotmail.fr and will expire on October 12, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-33196 - Server binary: /Users/tjaskula/.graphlab/anaconda/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1454972915.log
[INFO] GraphLab Server Version: 1.8.1


PROGRESS: Finished parsing file /Users/tjaskula/Documents/GitHub/Kaggle.HomeDepot/data/train.csv
PROGRESS: Parsing completed. Parsed 100 lines in 0.113239 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,int,str,str,float]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Finished parsing file /Users/tjaskula/Documents/GitHub/Kaggle.HomeDepot/data/train.csv
PROGRESS: Parsing completed. Parsed 74067 lines in 0.162205 secs.


In [3]:
test = gl.SFrame.read_csv("../data/test.csv")

PROGRESS: Finished parsing file /Users/tjaskula/Documents/GitHub/Kaggle.HomeDepot/data/test.csv
PROGRESS: Parsing completed. Parsed 100 lines in 0.204127 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,int,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Finished parsing file /Users/tjaskula/Documents/GitHub/Kaggle.HomeDepot/data/test.csv
PROGRESS: Parsing completed. Parsed 166693 lines in 0.328047 secs.


In [4]:
desc = gl.SFrame.read_csv("../data/product_descriptions.csv")

PROGRESS: Finished parsing file /Users/tjaskula/Documents/GitHub/Kaggle.HomeDepot/data/product_descriptions.csv
PROGRESS: Parsing completed. Parsed 100 lines in 0.493478 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Read 61134 lines. Lines per second: 60645.4
PROGRESS: Finished parsing file /Users/tjaskula/Documents/GitHub/Kaggle.HomeDepot/data/product_descriptions.csv
PROGRESS: Parsing completed. Parsed 124428 lines in 1.60894 secs.


### Data merging

In [5]:
# merge train with description
train = train.join(desc, on = 'product_uid', how = 'left')

In [6]:
# merge test with description
test = test.join(desc, on = 'product_uid', how = 'left')

### Let's explore some data

Let's examine 3 different queries and products:
* first from the training set
* somewhere in the moddle in the training set
* the last one from the training set

In [7]:
first_doc = train[0]
first_doc

{'id': 2,
 'product_description': 'Not only do angles make joints stronger, they also provide more consistent, straight corners. Simpson Strong-Tie offers a wide variety of angles in various sizes and thicknesses to handle light-duty jobs or projects where a structural connection is needed. Some can be bent (skewed) to match the project. For outdoor projects or those where moisture is present, use our ZMAX zinc-coated connectors, which provide extra resistance against corrosion (look for a "Z" at the end of the model number).Versatile connector for various 90 connections and home repair projectsStronger than angled nailing or screw fastening aloneHelp ensure joints are consistently straight and strongDimensions: 3 in. x 3 in. x 1-1/2 in.Made from 12-Gauge steelGalvanized for extra corrosion resistanceInstall with 10d common nails or #9 x 1-1/2 in. Strong-Drive SD screws',
 'product_title': 'Simpson Strong-Tie 12-Gauge Angle',
 'product_uid': 100001,
 'relevance': 3.0,
 'search_term': '

**'angle bracket'** search term is not contained in the body. **'angle'** would be after stemming however **'bracket'** is not.

In [8]:
middle_doc = train[37033]
middle_doc

{'id': 113228,
 'product_description': 'PureBond Plywood Project Panels are a convenient and cost-effective way to build cabinets, furniture and other woodworking projects. It provides a beautiful wood veneer face bonded to a strong and flat wood core. These PureBond Project Panels are made with no added formaldehyde, eliminating the concern about off-gassing dangerous fumes during fabrication or when installed in your home. Their smaller size makes them easy to handle and allows you to order just the amount of wood you need. PureBond plywood, in Project Panels sizes or in full sheet sizes, are a Home Depot exclusive.California residents: see&nbsp;Proposition 65 informationDecorative mahogany veneer applied to both sides of this panelB-2 plain sliced mahogany - 7-ply constructionLight weight, all-wood veneer constructionPrecision-cut hardwood plywood panels in convenient small sizesCommon: 3/4 in. x 2 ft. x 4 ft.; Actual: 0.703 in. x 24 in. x 48 in.Grade: B-2',
 'product_title': '3/4 i

only **'wood'** is present from search term

In [9]:
last_doc = train[-1]
last_doc

{'id': 221473,
 'product_description': 'No. 918 Millennial Ryan heathered texture semi-sheer curtain is a casual solid that adds freshness and a finishing touch to any decor setting. Enhances privacy while allowing light to gently filter through. Clean, simple one-pocket pole top design can be used with a standard or decorative curtain rod. Mix and match with other solids and prints for a look that is all your own.Sheer panel, gently filters lightNo header pole top panelMachine washableWide array of colors to choose from100% polyesterContains 1-curtain panel',
 'product_title': 'LICHTENBERG Pool Blue No. 918 Millennial Ryan Heathered Texture Sheer Curtain Panel, 40 in. W x 63 in. L',
 'product_uid': 206650,
 'relevance': 2.33,
 'search_term': 'fine sheer curtain 63 inches'}

**'sheer'** and **'courtain'** are present and that's all

### How many search terms are not present in description and title for ranked 3 documents
Ranked 3 documents are the most relevents searches, but how many search queries doesn't include the searched term in the description and the title

In [10]:
train['search_term_word_count'] = gl.text_analytics.count_words(train['search_term'])
ranked3doc = train[train['relevance'] == 3]
print ranked3doc.head()
len(ranked3doc)

+-----+-------------+-------------------------------+
|  id | product_uid |         product_title         |
+-----+-------------+-------------------------------+
|  2  |    100001   | Simpson Strong-Tie 12-Gaug... |
|  9  |    100002   | BEHR Premium Textured Deck... |
|  18 |    100006   | Whirlpool 1.9 cu. ft. Over... |
|  21 |    100006   | Whirlpool 1.9 cu. ft. Over... |
|  27 |    100009   | House of Fara 3/4 in. x 3 ... |
|  35 |    100011   | Toro Personal Pace Recycle... |
|  37 |    100011   | Toro Personal Pace Recycle... |
|  65 |    100016   | Sunjoy Calais 8 ft. x 5 ft... |
| 123 |    100023   | Quikrete 80 lb. Crack-Resi... |
| 162 |    100029   | DecoArt Americana Decor 16... |
+-----+-------------+-------------------------------+
+--------------------------------+-----------+-------------------------------+
|          search_term           | relevance |      product_description      |
+--------------------------------+-----------+-------------------------------+
|      

19125

In [11]:
words_search = gl.text_analytics.tokenize(ranked3doc['search_term'], to_lower = True)
words_description = gl.text_analytics.tokenize(ranked3doc['product_description'], to_lower = True)
words_title = gl.text_analytics.tokenize(ranked3doc['product_title'], to_lower = True)
wordsdiff_desc = []
wordsdiff_title = []
puid = []
search_term = []
ws_count = []
ws_count_used_desc = []
ws_count_used_title = []
for item in xrange(len(ranked3doc)):
    ws = words_search[item]
    pd = words_description[item]
    pt = words_title[item]
    diff = set(ws) - set(pd)
    if diff is None:
        diff = 0
    wordsdiff_desc.append(diff)
    
    diff2 = set(ws) - set(pt)
    if diff2 is None:
        diff2 = 0
    wordsdiff_title.append(diff2)
    
    puid.append(ranked3doc[item]['product_uid'])
    search_term.append(ranked3doc[item]['search_term'])
    ws_count.append(len(ws))
    ws_count_used_desc.append(len(ws) - len(diff))
    ws_count_used_title.append(len(ws) - len(diff2))
    
differences = gl.SFrame({"puid" : puid,
                         "search term": search_term,
                         "diff desc" : wordsdiff_desc,
                         "diff title" : wordsdiff_title,
                         "ws count" : ws_count, 
                         "ws count used desc" : ws_count_used_desc,
                         "ws count used title" : ws_count_used_title})

In [12]:
differences.sort(['ws count used desc', 'ws count used title'])

diff desc,diff title,puid,search term,ws count,ws count used desc
"[recycling, bins]","[recycling, bins]",145727,recycling bins,2,0
"[over, deck]","[over, deck]",100002,deck over,2,0
"[hammer, electric, drill]","[hammer, electric, drill]",120061,electric hammer drill,3,0
[microwaves],[microwaves],100006,microwaves,1,0
[plywoods],[plywoods],119996,plywoods,1,0
"[coca, cola]","[coca, cola]",120276,coca cola,2,0
[greenhouses],[greenhouses],120318,greenhouses,1,0
"[pipe, cutters]","[pipe, cutters]",119840,pipe cutters,2,0
"[buit, themostat, in]","[buit, themostat, in]",206359,buit in themostat,3,0
"[mowers, ridding]","[mowers, ridding]",120366,ridding mowers,2,0


In [13]:
print "No terms used in description : " + str(len(differences[differences['ws count used desc'] == 0]))
print "No terms used in title : " + str(len(differences[differences['ws count used title'] == 0]))
print "No terms used in description and title : " + str(len(differences[(differences['ws count used desc'] == 0) & 
                                                                        (differences['ws count used title'] == 0)]))

No terms used in description : 2666
No terms used in title : 2152
No terms used in description and title : 1206


In [14]:
import matplotlib.pyplot as plt
%matplotlib inline

### TF-IDF with linear regression

In [15]:
train_search_tfidf = gl.text_analytics.tf_idf(train['search_term_word_count'])

In [16]:
train['search_tfidf'] = train_search_tfidf

In [17]:
train['product_desc_word_count'] = gl.text_analytics.count_words(train['product_description'])
train_desc_tfidf = gl.text_analytics.tf_idf(train['product_desc_word_count'])

In [18]:
train['desc_tfidf'] = train_desc_tfidf

In [19]:
train['product_title_word_count'] = gl.text_analytics.count_words(train['product_title'])
train_title_tfidf = gl.text_analytics.tf_idf(train['product_title_word_count'])
train['title_tfidf'] = train_title_tfidf

In [20]:
train['distance_desc'] = train.apply(lambda x: gl.distances.cosine(x['search_tfidf'],x['desc_tfidf']))
train['distance_desc_sqrt'] = train['distance_desc'] ** 2
train['distance_title'] = train.apply(lambda x: gl.distances.cosine(x['search_tfidf'],x['title_tfidf']))
train['distance_title_sqrt'] = train['distance_title'] ** 3

In [21]:
model1 = gl.linear_regression.create(train, target = 'relevance', 
                                         features = ['distance_desc', 'distance_desc_sqrt', 'distance_title', 'distance_title_sqrt'], 
                                         validation_set = None)

PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 74067
PROGRESS: Number of features          : 4
PROGRESS: Number of unpacked features : 4
PROGRESS: Number of coefficients    : 5
PROGRESS: Starting Newton Method
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: | 1         | 2        | 1.045566     | 1.754684           | 0.509505      |
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: SUCCESS: Optimal solution found.
PROGRESS:


In [22]:
#let's take a look at the weights before we plot
model1.get("coefficients")

name,index,value,stderr
(intercept),None,2.53757638531,0.0658468389666
distance_desc,None,1.80762596819,0.169354041006
distance_desc_sqrt,None,-1.42719146154,0.104718192476
distance_title,None,-0.93309498412,0.0420113779652
distance_title_sqrt,None,0.170496189237,0.025047571535


In [23]:
test['search_term_word_count'] = gl.text_analytics.count_words(test['search_term'])
test_search_tfidf = gl.text_analytics.tf_idf(test['search_term_word_count'])
test['search_tfidf'] = test_search_tfidf
test['product_desc_word_count'] = gl.text_analytics.count_words(test['product_description'])
test_desc_tfidf = gl.text_analytics.tf_idf(test['product_desc_word_count'])
test['desc_tfidf'] = test_desc_tfidf
test['product_title_word_count'] = gl.text_analytics.count_words(test['product_title'])
test_title_tfidf = gl.text_analytics.tf_idf(test['product_title_word_count'])
test['title_tfidf'] = test_title_tfidf

test['distance_desc'] = test.apply(lambda x: gl.distances.cosine(x['search_tfidf'],x['desc_tfidf']))
test['distance_desc_sqrt'] = test['distance_desc'] ** 2
test['distance_title'] = test.apply(lambda x: gl.distances.cosine(x['search_tfidf'],x['title_tfidf']))
test['distance_title_sqrt'] = test['distance_title'] ** 3

In [24]:
'''
predictions_test = model1.predict(test)
test_errors = predictions_test - test['relevance']
RSS_test = sum(test_errors * test_errors)
print RSS_test
'''

"\npredictions_test = model1.predict(test)\ntest_errors = predictions_test - test['relevance']\nRSS_test = sum(test_errors * test_errors)\nprint RSS_test\n"

In [25]:
predictions_test = model1.predict(test)
predictions_test

dtype: float
Rows: 166693
[2.199192361518506, 2.1554120970764274, 2.3493517485733406, 2.3541822792039477, 2.313897641296237, 2.1554120970764274, 2.3777272648703045, 2.386769665285134, 2.183135953769775, 2.6506602777391195, 2.5108080126720074, 2.396619471680039, 2.5046996384663696, 2.3343196365927374, 2.2415627071801802, 2.4471028886950865, 2.1554120970764274, 2.4948525445256777, 2.2110612681180215, 2.2696507298175015, 2.688643049230895, 2.595215641273066, 2.225188575392107, 2.3908436769509738, 2.1554120970764274, 2.3000724771559944, 2.1554120970764274, 2.374087054859538, 2.2756016442054774, 2.35964405581698, 2.519301460479922, 2.415486507452863, 2.2697683214326454, 2.3884126045344267, 2.250411773743458, 2.1554120970764274, 2.2714197565820995, 2.2071881132454934, 2.684611941462438, 2.589197435706547, 2.1985125186230863, 2.3432345144738527, 2.2742884640490084, 2.1554120970764274, 2.5023523052781433, 2.4254295602988183, 2.472238345284125, 2.4579697258335824, 2.6624497149352377, 2.30717273

In [26]:
submission = gl.SFrame(test['id'])

In [27]:
submission.add_column(predictions_test)
submission.rename({'X1': 'id', 'X2':'relevance'})

id,relevance
1,2.19919236152
4,2.15541209708
5,2.34935174857
6,2.3541822792
7,2.3138976413
8,2.15541209708
10,2.37772726487
11,2.38676966529
12,2.18313595377
13,2.65066027774


In [28]:
submission['relevance'] = submission.apply(lambda x: 3.0 if x['relevance'] > 3.0 else x['relevance'])
submission['relevance'] = submission.apply(lambda x: 1.0 if x['relevance'] < 1.0 else x['relevance'])

In [29]:
submission['relevance'] = submission.apply(lambda x: str(x['relevance']))

In [30]:
submission.export_csv('../data/submission2.csv', quote_level = 3)

In [ ]:
#gl.canvas.set_target('ipynb')